# Exploratory Data Analysis - Coffee Tree

In [2]:
import pandas as pd
from janitor import clean_names

df = clean_names(pd.read_excel("data/raw/modelo_cafe_2024out.xlsm", nrows=627))

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 41 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   laboratorio                               626 non-null    object 
 1   amostra                                   626 non-null    object 
 2   odor_do_po                                626 non-null    float64
 3   odor_da_bebida_*                          626 non-null    float64
 4   sabor_da_bebida_*                         626 non-null    float64
 5   _docura                                   626 non-null    float64
 6   _corpo                                    626 non-null    float64
 7   _acidez_intensidade                       626 non-null    float64
 8   _acidez_qualidade                         626 non-null    float64
 9   _amargor                                  626 non-null    float64
 10  _adstringencia                        

## Observações da Aline

Interessante não cnsiderar esses atributos:         
    - `odor_da_bebida_*`;       
    - `sabor_da_bebida_*`;     

Target:     
    - `qualidade_global_media_dada_pela_equipe_`;     

Identificação das amostras:     
    - `laboratorio`;        
    - `amostra`;        
    - `fora_de_tipo_x_tipo_unico`;          
    - `categoria`;      
    - `sigla_categoria`;        

Médias dos atributos:       
    - `odor_do_po`;     
    - `_docura;`;    
    - `_corpo`;     
    - `_acidez_intensidade`;     
    - `_acidez_qualidade`;     
    - `_amargor`;     
    - `_adstringencia`;     
    - `_intensidade`;      

Mediana dos descritores:        
    - `_alcoolico_`;     
    - `_cedro_carvalho_amadeirado_`;     
    - `_amendoado_castanhas_`;     
    - `_animalico_curral_`;     
    - `_azedo_`;     
    - `_baunilha_`;     
    - `_borracha_`;     
    - `_caramelizado_caramelo_doce_`;     
    - `_cereal_`;     
    - `_chocolate_cacau_`;     
    - `_cozido_assado_`;     
    - `_especiarias_`;     
    - `_fermentado_`;     
    - `_floral_`;     
    - `_frutado_`;     
    - `_iodoformio_quimico_`;     
    - `_madeira_papelao_`;     
    - `_mel_`;     
    - `_queimado_defumado_`;     
    - `_terroso_mofo_`;     
    - `_tostado_`;     
    - `_vegetal_`;     
    - `_velho_oxidado_`;     
    - `_verde_herbaceo_`;     




# Análise de correlação

In [3]:
df[
    [
        "odor_da_bebida_*",
        "sabor_da_bebida_*",
        "qualidade_global_media_dada_pela_equipe_",
    ]
].corr()

,odor_da_bebida_*,sabor_da_bebida_*,qualidade_global_media_dada_pela_equipe_
odor_da_bebida_*,1.000000,0.988710,0.981344
sabor_da_bebida_*,0.988710,1.000000,0.990458
qualidade_global_media_dada_pela_equipe_,0.981344,0.990458,1.000000


# Seleção de atributos

In [4]:
df_model = df[
    [
        # Target
        "qualidade_global_media_dada_pela_equipe_",
        # Classificação de laboratórios
        "laboratorio",
        "fora_de_tipo_x_tipo_unico",
        # Médias dos atributos
        "odor_do_po",
        "_docura",
        "_corpo",
        "_acidez_intensidade",
        "_acidez_qualidade",
        "_amargor",
        "_adstringencia",
        "_intensidade",
        # Mediana dos descritores
        "_alcoolico_",
        "_cedro_carvalho_amadeirado_",
        "_amendoado_castanhas_",
        "_animalico_curral_",
        "_azedo_",
        "_baunilha_",
        "_borracha_",
        "_caramelizado_caramelo_doce_",
        "_cereal_",
        "_chocolate_cacau_",
        "_cozido_assado_",
        "_especiarias_",
        "_fermentado_",
        "_floral_",
        "_frutado_",
        "_iodoformio_quimico_",
        "_madeira_papelao_",
        "_mel_",
        "_queimado_defumado_",
        "_terroso_mofo_",
        "_tostado_",
        "_vegetal_",
        "_velho_oxidado_",
        "_verde_herbaceo_",
    ]
]

# Análise de multicolinearidade

In [5]:
# Análise de VIF (Variance Inflation Factor)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Separar features e target, usando apenas variáveis numéricas
X = df_model.select_dtypes(include=["float64", "int64"]).drop(
    "qualidade_global_media_dada_pela_equipe_", axis=1
)
y = df_model["qualidade_global_media_dada_pela_equipe_"]

# Calcular VIF para cada feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Ordenar por VIF em ordem decrescente
vif_data = vif_data.sort_values("VIF", ascending=False)

# Mostrar resultados
print("Fatores de Inflação da Variância (VIF):")
print("VIF > 10 indica alta multicolinearidade")
print("\n", vif_data)

Fatores de Inflação da Variância (VIF):
VIF > 10 indica alta multicolinearidade

                          Feature          VIF
4              _acidez_qualidade  1333.365775
3            _acidez_intensidade  1216.629423
5                       _amargor  1044.731088
2                         _corpo   919.100115
6                 _adstringencia   895.623322
7                   _intensidade   625.354660
0                     odor_do_po   353.212952
1                        _docura   267.123491
22                     _frutado_     8.789442
24             _madeira_papelao_     6.395278
15  _caramelizado_caramelo_doce_     5.544807
30               _velho_oxidado_     4.730044
26           _queimado_defumado_     4.141044
10         _amendoado_castanhas_     3.234078
17             _chocolate_cacau_     2.916983
28                     _tostado_     2.196520
25                         _mel_     1.917277
14                    _borracha_     1.697078
31              _verde_herbaceo_     1.45577

## Interpretação dos resultados

1. **Variáveis com alta multicolinearidade (VIF > 10)**:
   - As variáveis **_acidez_qualidade** (1333.37), **_acidez_intensidade** (1216.63) e **_amargor** (1044.73) possuem VIF extremamente altos, indicando que essas variáveis estão fortemente correlacionadas entre si ou com outras variáveis do modelo. Essa multicolinearidade pode fazer com que seja difícil distinguir o efeito individual de cada uma dessas variáveis sobre o desfecho, e pode causar instabilidade nos coeficientes estimados.
   - O **_corpo** (919.10) e **_adstringencia** (895.62) também possuem VIF muito elevados, apontando para multicolinearidade significativa.

2. **Variáveis com VIF entre 5 e 10**:
   - Essas variáveis, como **_frutado_** (8.79), **_madeira_papelao_** (6.40), e **_caramelizado_caramelo_doce_** (5.54), apresentam uma multicolinearidade moderada. Embora não tão grave quanto as primeiras, essas variáveis ainda podem afetar a precisão das estimativas e devem ser monitoradas, pois um VIF acima de 5 já é considerado uma indicação de atenção.

3. **Variáveis com baixa multicolinearidade (VIF < 5)**:
   - A maioria das variáveis restantes, como **_velho_oxidado_** (4.73), **_queimado_defumado_** (4.14), e **_tostado_** (2.20), apresenta VIFs abaixo de 5, o que indica pouca ou nenhuma multicolinearidade preocupante. Estas variáveis estão relativamente bem comportadas no modelo.


# Salvar o dataframe limpo


In [ ]:
df_model.to_parquet("data/processed/df_model.parquet")